In [2]:
import pandas as pd

In [11]:
sales_df = pd.read_csv("data_avance/sales.csv")
products_df = pd.read_csv("data_avance/products.csv")


In [12]:
print("Ventas (sales):")
display(sales_df.head())

print("Productos (products):")
display(products_df.head())

Ventas (sales):


,SalesID,SalesPersonID,CustomerID,ProductID,Quantity,Discount,TotalPrice,SalesDate,TransactionNumber
0,1,6,27039,381,7,0.0,0.0,2018-02-05 07:38:25.430,FQL4S94E4ME1EZFTG42G
1,2,16,25011,61,7,0.0,0.0,2018-02-02 16:03:31.150,12UGLX40DJ1A5DTFBHB8
2,3,13,94024,23,24,0.0,0.0,2018-05-03 19:31:56.880,5DT8RCPL87KI5EORO7B0
3,4,8,73966,176,19,0.2,0.0,2018-04-07 14:43:55.420,R3DR9MLD5NR76VO17ULE
4,5,10,32653,310,9,0.0,0.0,2018-02-12 15:37:03.940,4BGS0Z5OMAZ8NDAFHHP3


Productos (products):


,ProductID,ProductName,Price,CategoryID,Class,ModifyDate,Resistant,IsAllergic,VitalityDays
0,1,Flour - Whole Wheat,74.2988,3,Medium,21:49.2,Durable,Unknown,0
1,2,Cookie Chocolate Chip With,91.2329,3,Medium,39:11.0,Unknown,Unknown,0
2,3,Onions - Cippolini,9.1379,9,Medium,11:51.6,Weak,FALSE,111
3,4,Sauce - Gravy; Au Jus; Mix,54.3055,9,Medium,46:28.9,Durable,Unknown,0
4,5,Artichokes - Jerusalem,65.4771,2,Low,13:35.4,Durable,TRUE,27


In [ ]:
# agrega el precio a cada venta segun su producto

merged_df = sales_df.merge(products_df[['ProductID', 'Price']], on='ProductID', how='left')

merged_df['TotalPriceCalculated'] = (merged_df['Quantity'] * merged_df['Price']) * (1 - merged_df['Discount'])

display(merged_df[['Quantity', 'Price', 'Discount', 'TotalPriceCalculated']].head())

,Quantity,Price,Discount,TotalPriceCalculated
0,7,44.2337,0.0,309.63590
1,7,62.5460,0.0,437.82200
2,24,79.0184,0.0,1896.44160
3,19,81.3167,0.2,1236.01384
4,9,79.9780,0.0,719.80200


In [18]:
import os

os.makedirs('data_avance', exist_ok=True)  # crea la carpeta si no existe

merged_df.to_csv('data_avance/sales_with_totalprice.csv', index=False)

In [ ]:
#Detecta los outliers en la columna de ventas totales (TotalPriceCalculated)
#Utilizando el criterio del rango intercuartílico (IQR). 
#Luego, crea una nueva columna llamada IsOutlier que tenga el valor 1 si el registro es un outlier y 0 en caso contrario. ¿Cuántos outliers se detectaron?

In [19]:
# calculamos los Q 
Q1 = merged_df['TotalPriceCalculated'].quantile(0.25)
Q3 = merged_df['TotalPriceCalculated'].quantile(0.75)
IQR = Q3 - Q1

In [20]:
# calculamos los límites
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [21]:
merged_df['IsOutlier'] = ((merged_df['TotalPriceCalculated'] < lower_bound) | (merged_df['TotalPriceCalculated'] > upper_bound)).astype(int)
#creamos la columna

In [ ]:
# contamos los outliers detectados
num_outliers = merged_df['IsOutlier'].sum()

print(f"Se detectaron {num_outliers} outliers en la columna TotalPriceCalculated.")

Se detectaron 48217 outliers en la columna TotalPriceCalculated.


In [25]:
#convertimos a datetime
merged_df['SalesDate'] = pd.to_datetime(merged_df['SalesDate'])

merged_df['SalesHour'] = merged_df['SalesDate'].dt.time #creamos columna 

ventas_por_hora = merged_df.groupby('SalesHour')['TotalPriceCalculated'].sum().reset_index()

merged_df['DayOfWeek'] = merged_df['SalesDate'].dt.dayofweek #identificamos dia de la semana

merged_df['DiaTipo'] = merged_df['DayOfWeek'].apply(lambda x: 'Entre semana' if x < 5 else 'Fin de semana')

ventas_por_diatipo = merged_df.groupby('DiaTipo')['TotalPriceCalculated'].sum().reset_index()

In [ ]:
# resultados
print("Ventas totales por hora:")
print(ventas_por_hora.sort_values(by='TotalPriceCalculated', ascending=False).head())

print("\nVentas totales según tipo de día:")
print(ventas_por_diatipo)

Ventas totales por hora:
               SalesHour  TotalPriceCalculated
339177   01:44:43.550000            8872.35248
2410173  12:25:02.410000            8531.76838
1143727  05:53:35.240000            8254.73550
3203701  16:30:29.220000            8196.92370
2232527  11:30:15.470000            7948.48490

Ventas totales según tipo de día:
         DiaTipo  TotalPriceCalculated
0   Entre semana          3.080352e+09
1  Fin de semana          1.235916e+09


In [ ]:
#La mayoría de las ventas ocurren en las horas de la mañana, entre las 7 y 11am, también el total de ventas es mayor entre semana que durante le fin de semana.

In [30]:
employees_df = pd.read_csv("data_avance/employees.csv")



In [33]:
employees_df['BirthDate'] = pd.to_datetime(employees_df['BirthDate'])
employees_df['HireDate'] = pd.to_datetime(employees_df['HireDate'])
merged_df['SalesDate'] = pd.to_datetime(merged_df['SalesDate'])



In [36]:
merged_df = merged_df.merge(
    employees_df[['EmployeeID', 'BirthDate', 'HireDate']],
    left_on='SalesPersonID',
    right_on='EmployeeID',
    how='left'
)

In [ ]:
# edad al momento de la contratacion
merged_df['AgeAtHire'] = (merged_df['HireDate'] - merged_df['BirthDate']).dt.days // 365

# años de experiencia cuando se hizo la venta
merged_df['ExperienceAtSale'] = (merged_df['SalesDate'] - merged_df['HireDate']).dt.days // 365

display(merged_df[['SalesPersonID', 'EmployeeID', 'BirthDate', 'HireDate', 'SalesDate', 'AgeAtHire', 'ExperienceAtSale']].head())

,SalesPersonID,EmployeeID,BirthDate,HireDate,SalesDate,AgeAtHire,ExperienceAtSale
0,6,6,1987-01-13,2013-06-22 13:20:18.080,2018-02-05 07:38:25.430,26,4.0
1,16,16,1951-07-07,2017-02-10 11:21:26.650,2018-02-02 16:03:31.150,65,0.0
2,13,13,1963-04-18,2011-12-12 10:43:52.940,2018-05-03 19:31:56.880,48,6.0
3,8,8,1956-12-13,2014-10-14 23:12:53.420,2018-04-07 14:43:55.420,57,3.0
4,10,10,1963-12-30,2012-07-23 15:02:12.640,2018-02-12 15:37:03.940,48,5.0


In [ ]:
# Muestran que los empleados fueron contratados a diferentes edades, desde jovenes hasta personas mayores.Tambien muestra su experiencia en la empresa al hacer las ventas varía mucho.

In [40]:
# seleccionamos colunas
df_model = merged_df[[
    'SalesID',
    'SalesPersonID',
    'CustomerID',
    'ProductID',
    'Quantity',
    'Discount',
    'SalesHour',
    'DiaTipo',           
    'AgeAtHire',
    'ExperienceAtSale',
    'IsOutlier',
    'TotalPriceCalculated'  
]].copy()

In [ ]:
# DiaTipo  0=Entre semana 1=Fin de semana

df_model['DiaTipo'] = df_model['DiaTipo'].map({'Entre semana': 0, 'Fin de semana': 1})

In [44]:
cols_to_scale = ['Quantity', 'Discount', 'AgeAtHire', 'ExperienceAtSale']

for col in cols_to_scale:
    mean = df_model[col].mean()
    std = df_model[col].std()
    df_model[col] = (df_model[col] - mean) / std

In [ ]:
# fueron escaladas utilizando la estandarización (normalización z-score), que consiste en restar la media y dividir por la desviación estándar de cada variable. 
# Esto permite que todas las variables numéricas tengan una escala comparable.
